# Concurrency

Definitions of concurrency:

* Concurrency is not the same as parallelism. 

* Concurrency is not a matter of application implementation. 

* Concurrency can be defined as follows: "Two events are concurrent if neither can causally affect the other."

* In other words, something is concurrent if it can be fully or partially decomposed into components (units) that are order-independent.

Common application scenarios where concurrent processing is a viable approach:

* Processing distribution: The scale of the problem is so big that the only way to process it in an acceptable time frame (with constrained resources) is to distribute execution on multiple processing units that can handle the work in parallel.

* Application responsiveness: Your application needs to maintain responsiveness (accept new inputs), even if it did not finish processing previous inputs.

* Background processing: Not every task needs to be performed in a synchronous way. If there is no need to immediately access the results of a specific action, it may be reasonable to defer execution in time.

Python's tools to deal with concurrency:

* Multithreading: 
    * Running multiple threads of execution that share the memory context of the parent process. 
    * The execution of threads is coordinated by the OS kernel.
    * It works best in applications that do a lot of I/O operations or need to maintain UI responsiveness. 
    * Very lightweight, but comes with caveats and memory safety risks.

* Multiprocessing: 
    * Running multiple independent processes to perform work in a distributed manner. 
    * Similar to threads in operation, but there's no shared memory context. 
    * Due to Python's GIL limitaton, it's better suited for CPU-intensive applications. 
    * More heavyweight than multithreading and requires implementing inter-process communication patterns to orchestrate work between processes.

* Asynchronous programming: 
    * Running multiple cooperative tasks within a single application process. 
    * Cooperative tasks work like threads, but switching between them is done by the application, not the OS kernel. 
    * Well suited to I/O-bound applications, especially for programs that need to handle multiple simultaneous network connections. 
    * The downside of asynchronous programming is the need to use dedicated asynchronous libraries.



### Multithreading

To start a new thread in Python, use the `threading.Thread()` class, as follows:


In [1]:
!python _01_multithreading/start_new_thread.py


Starting thread...
printing from thread
Ending thread...


From the last code example:

* To start a new thread, we call the `start()` method.

* Once the new thread is started, it'll run next to the main thread until the target function finishes. 

* To end the thread, we wait for the thread to finish with the `join()` method, which is a blocking operation.


With a small modification, we can also start and join multiple threads in bulk, as follows:


In [2]:
!python _01_multithreading/start_new_threads.py


Starting threads...

printing from thread
printing from thread
printing from threadprinting from thread

printing from thread
printing from thread
printing from thread
printing from threadprinting from thread

printing from thread
Ending threads...


Thread safety:

* All threads share the same memory context, so we must be careful when many threads access the same data structures. 

* If 2 parallel threads update the same variable without any protection, there might be a situation where a subtle timing variation in thread execution can alter the final result in an unexpected way. 

* Such situations are called `race conditions`, and they're very hard to debug. In those cases, the code is not 'thread-safe'.

For example, the following program is not thread-safe, and it returns a different value in every execution:


In [3]:
# The correct output is 100 threads * 100.000 iterations = 10.000.000

!python _01_multithreading/thread_visits.py 


thread_count=100, thread_visits=9361647


To avoid race conditions, we need to to use thread locking primitives. Python provides the `threading.Lock` class, which is a simple implementation of a thread lock. 

The following is an example of a thread-safe variant of the last code:



In [4]:
!python _01_multithreading/thread_safe_visits.py


thread_count=100, thread_visits=10000000


In the last code example, the thread visits with locks are counted properly, but at the expense of lower performance: 

* The lock will make sure that only 1 thread at a time can process a single block of code, so the protected block cannot run in parallel. 

* Also, acquiring and releasing locks are operations that add overhead.


#### Python's limitation with threads:

The standard implementation of Python (the CPython interpreter) comes with a major limitation that renders threads less useful in many contexts: 

* All operations accessing Python objects are serialized by the `Global Interpreter Lock (GIL)`.

* This is done because many of the interpreter's internal structures are not thread-safe and need to be protected. 

* Not every operation requires locking, and there are certain situations when threads release the lock:
    * In blocking system calls like socket calls.     
    * In sections of C extensions that do not use any Python/C API functions.

* So multiple threads can do I/O operations or execute some C extension code completely in parallel.


### A simple multithreaded application

Consider a program that fetches foreign exchange rates from an external source:

* The exchange rates are available from a free API at `https://www.vatcomply.com`. 

* We want to obtain exchange rates for multiple currencies and present the results as an exchange rate currency matrix.

* But the API doesn't allow us to query for data using multiple base currencies at once, so we need to fetch them one by one.

The following is a naive synchronous solution that doesn't use threads at all:


In [5]:
!python _02_example_threaded_application/synchronous.py


1 USD =     1.0 USD,   0.912 EUR,    3.96 PLN,    10.4 NOK,    22.4 CZK
1 EUR =     1.1 USD,     1.0 EUR,    4.34 PLN,    11.4 NOK,    24.5 CZK
1 PLN =   0.253 USD,   0.231 EUR,     1.0 PLN,    2.62 NOK,    5.65 CZK
1 NOK =  0.0964 USD,   0.088 EUR,   0.382 PLN,     1.0 NOK,    2.16 CZK
1 CZK =  0.0447 USD,  0.0408 EUR,   0.177 PLN,   0.463 NOK,     1.0 CZK

time elapsed: 2.93s


The last code example is mostly I/O bound, so we can improve it with multithreading. The simplest approach is to use one thread per parameter value (defined in `SYMBOLS`) , as follows:


In [6]:
!python _03_one_thread_per_item/one_thread_per_item.py


1 NOK =  0.0964 USD,   0.088 EUR,   0.382 PLN,     1.0 NOK,    2.16 CZK
1 EUR =     1.1 USD,     1.0 EUR,    4.34 PLN,    11.4 NOK,    24.5 CZK
1 CZK =  0.0447 USD,  0.0408 EUR,   0.177 PLN,   0.463 NOK,     1.0 CZK
1 PLN =   0.253 USD,   0.231 EUR,     1.0 PLN,    2.62 NOK,    5.65 CZK
1 USD =     1.0 USD,   0.912 EUR,    3.96 PLN,    10.4 NOK,    22.4 CZK

time elapsed: 0.86s


The last code example shows substantial improvement. But there are issues with it:

* We start a new thread for every parameter:

    * Thread initialization takes time and consume resources like memory or file descriptors. 
    * Our example input has a small number of items, but it's not a good idea to run an unbound number of threads that depend on the size of data input.

* The `fetch_rates()` function calls the built-in `print()` function:

    * `print()` has issues due to the way the standard output is buffered in Python, which may cause malformed output when multiple function calls interweave between threads.
    * `print()` is slow; so if used in many threads, it can lead to slow function executions that will erase any gains from multithreading.

* By delegating every function call to a separate thread, it's hard to control the rate of network calls:

    * External services enforce limits on the rate of requests from a single client. 
    * So it's a good idea to apply throttling to the rate of processing, to avoid being blacklisted by APIs for abusing their usage limits.

We can fix the 1st and 3rd issue with thread pools:

* Thread pools start a predefined number of threads that will consume the work items from a queue until it becomes empty. 

* When there is no more work to do, the threads will quit, so we'll be able to exit from the program. 

* The `Queue` class from the `queue` module can be used. It's a thread-safe FIFO queue implementation. 

The following is a modified versionof the last code that uses a thread pool:



In [7]:
!python _04_Using_thread_pool/thread_pool.py


1 NOK =  0.0964 USD,   0.088 EUR,   0.382 PLN,     1.0 NOK,    2.16 CZK1 EUR =     1.1 USD,     1.0 EUR,    4.34 PLN,    11.4 NOK,    24.5 CZK

1 PLN =   0.253 USD,   0.231 EUR,     1.0 PLN,    2.62 NOK,    5.65 CZK
1 USD =     1.0 USD,   0.912 EUR,    3.96 PLN,    10.4 NOK,    22.4 CZK
1 CZK =  0.0447 USD,  0.0408 EUR,   0.177 PLN,   0.463 NOK,     1.0 CZK

time elapsed: 1.16s


The modified function still has the 2nd issue unsolved: the malformed output when 2 threads attempt to print results at the same time. That can bee fixed by using 2-way queues:

* We use another queue responsible for collecting results from our workers. 

* Then the main threadprints the results from the 2nd queue.

The following is the modified code that uses 2-way queues:


In [11]:
!python _05_Using_2way_queues/two_way_queues.py


1 EUR =     1.1 USD,     1.0 EUR,    4.34 PLN,    11.4 NOK,    24.5 CZK
1 NOK =  0.0964 USD,   0.088 EUR,   0.382 PLN,     1.0 NOK,    2.16 CZK
1 PLN =   0.253 USD,   0.231 EUR,     1.0 PLN,    2.62 NOK,    5.65 CZK
1 CZK =  0.0447 USD,  0.0408 EUR,   0.177 PLN,   0.463 NOK,     1.0 CZK
1 USD =     1.0 USD,   0.912 EUR,    3.96 PLN,    10.4 NOK,    22.4 CZK

time elapsed: 1.15s
